# Part 1: Importing Necessary Libraries

In [14]:
import google.generativeai as genai
from IPython.display import Markdown

import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import matplotlib.pyplot as plt

import os   
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import random

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
import webbrowser
from json.decoder import JSONDecodeError

# pytest
import pytest
import unittest
from unittest.mock import patch, Mock

/Users/juliaepshtein/Desktop/520_Final_Project/Personify/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
load_dotenv()
    
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI = os.getenv("REDIRECT_URI")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

# Part 2: Logging in With Authentication

In [16]:
# Create App With User Authentication

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
redirect_uri=REDIRECT_URI,
scope="user-library-read user-top-read"))

In [17]:
# Get top 50 tracks from the medium term (last 6 months)
top_tracks = sp.current_user_top_tracks(limit=50, time_range='short_term')

track_info = []
for item in top_tracks['items']:
    track_name = item["name"]
    artist_name = item["artists"][0]["name"]

    try:
        artist = sp.search(q=artist_name, type="artist")["artists"]["items"][0]
        genres = artist["genres"]
    except IndexError:
        genres = []

    track_info.append({
        "track": track_name,
        "artist": artist_name,
        "genres": genres
    })

df = pd.DataFrame(track_info)
df.head()

,track,artist,genres
0,No One Noticed,The Marías,[bedroom pop]
1,Fire for You,Cannons,[]
2,Echo,The Marías,[bedroom pop]
3,Run Your Mouth,The Marías,[bedroom pop]
4,Give You the World,Steve Lacy,[]


In [18]:
tracks_text = "\n".join([f"{t['track']} by {t['artist']}" for t in track_info])

print("Preview of tracks sent to GPT:\n")

print(tracks_text[:1000])

Preview of tracks sent to GPT:

No One Noticed by The Marías
Fire for You by Cannons
Echo by The Marías
Run Your Mouth by The Marías
Give You the World by Steve Lacy
URRRGE!!!!!!!!!! (feat. A$AP Rocky) by Doja Cat
The Basement by Lunar Vacation
Care For You by The Marías
BEST INTEREST by Tyler, The Creator
Be Careful With Me by Omar Apollo
All Night by Men I Trust
Ojos Tristes (with The Marías) by Selena Gomez
Words by Gregory Alan Isakov
Amoeba by Clairo
Constant Repeat by Charli xcx
Thank You by Clairo
Lejos de Ti by The Marías
Pool by Still Woozy
Roof by Rico Nasty
Spite by Omar Apollo
Superstar Sh*t by Dominic Fike
7 Hours by Dominic Fike
Hey Girl by Paul Cherry
Demons by Doja Cat
Psycho Pass by Xavier Wulf
Strawberry Skies by Kid Travis
Hoe Cakes by MF DOOM
Cool About It by boygenius
Maybe She by Still Woozy
angel by Kali Uchis
Intergalactic Janet by Ley Soul
OMG by Suki Waterhouse
what u need (angelica's place) by Chase Shakur
Tsunami by Suzuki Saint
Better People To Leave On Rea

In [ ]:
generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,
}

In [22]:
vibe_model = genai.GenerativeModel('gemini-1.5-flash')

def get_top_vibes(track_list, top_n=3):
    """Analyze a list of tracks and return dominant vibes"""
    # Prepare track information for Gemini
    track_summary = "\n".join([
        f"- {track['name']} by {track['artists'][0]['name']}" 
        for track in track_list['items']
    ])
    
    prompt = f"""
    Analyze these songs from my listening history and identify the dominant vibes:
    {track_summary}
    
    Possible vibe categories (choose up to {top_n} dominant ones):
    - Chill 
    - Energetic 
    - Melancholy
    - Romantic
    - Confident
    - Nostalgic
    - Artsy
    - Dark
    - Rage
    - Futuristic
    - Party
    - Ambient
    - Spiritual
    - Dreamy
    - Rebellious
    - Carefree
    - Classy
    - Cinematic
    - Theatrical
    - Alternative
    
    Provide only the {top_n} most dominant vibe categories as a comma-separated list (no explanations).
    Example: "Energetic/Upbeat, Chill/Relaxed, Emotional/Introspective"
    """
    
    try:
        response = vibe_model.generate_content(prompt)
        return [vibe.strip() for vibe in response.text.split(",")[:top_n]]
    except Exception as e:
        print(f"Error analyzing vibes: {e}")
        return ["Unknown"]

# Get top tracks and analyze overall vibes
top_tracks = sp.current_user_top_tracks(limit=50, time_range='short_term')
dominant_vibes = get_top_vibes(top_tracks)

print("\nYour dominant music vibes:")
for i, vibe in enumerate(dominant_vibes, 1):
    print(f"{i}. {vibe}")


Your dominant music vibes:
1. Chill/Relaxed
2. Dreamy/Ethereal
3. Emotional/Introspective
